In [6]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tifffile as tf
from scipy.ndimage.filters import gaussian_filter1d as gaussian
from glob import glob
from scipy.signal import savgol_filter, medfilt
import matplotlib.ticker as ticker
from scipy.optimize import curve_fit

In [2]:
# 1. load data into pandas data frame

# 2. convert data from I(t) to c_i(t) via equation for G quadruplex binding
    # - find t0
    # - 
# 3. calculate dc_i/dt for each GUV

# 4. calculate background intensity and convert this to c_0(t) via g quadruplex binding model

# 5. plot dc_/dt/A_GUV vs c_0-c_i (expect to see a positive linear relationship)


# as data includes actual G4 background for GUV permeability and later Gramicidin A experiments, start with these experiments


In [3]:
App = 3.16 #um^2

# G quadruplex model binding


def exp_I(c, k = 8.8,A = 0.39):
    return  A+ (1-A)/(k_1*c**2+1)

def inv(I, k = 8.8,A = 0.39):
    
    if I > 1:
        return 0
    
    elif I < A:
        return 1
        
    else:
        return np.sqrt((1/k)*((1-A)/(I-A)-1))


#smoothing configs

sigma = 30




# Calibration model fit


## **If you have already fitted a calibration function and saved the parameters skip this section**


In [5]:
# finding a suitable calibration

calib_data = pd.read_csv('../20210323_KCLFAMG4Q0to20mMCalib/RelIntensG41Concentration.csv')

In [6]:
raw_data = pd.read_csv('../20210323_KCLFAMG4Q0to20mMCalib/1uM/Intensities0-2mM.csv')
n = 512**2

print(raw_data['StdDev']/np.sqrt(n))

dI = raw_data['StdDev']/np.sqrt(n)
drelI = dI/raw_data['Mean']

drelI = drelI*10

rel_errorK = 0.02


0    2.244234
1    2.140793
2    1.921438
3    1.709535
4    1.638162
5    1.617246
Name: StdDev, dtype: float64


In [7]:
%matplotlib qt
I= np.linspace(1,0.2,100)

fig, ax = plt.subplots(1,1)

ax.scatter(calib_data['Rel. Intes'],calib_data['K conc [mM]'],marker = '+',s = 5)

legend = []

# fit a polynomial


fit = np.polyfit(-np.log(calib_data['Rel. Intes'].iloc[:-1]),calib_data['K conc [mM]'].iloc[:-1],3)

def fit_func(I,fit):
    
    c = np.zeros_like(I)
    
    for i in range(1,len(fit)+1):

        c += fit[i-1]*(-np.log(I))**(len(fit)-i)
        print(len(fit)-i)
    return c



ax.plot(I,fit_func(I,fit))

legend.append('Calibration')

def model_fit(cK,kb,c):
    return c + (1-c)/(kb*cK**2+1)

params = curve_fit(model_fit,calib_data['K conc [mM]'],calib_data['Rel. Intes'], p0 = [8.8,0.45])

cK_bar = np.linspace(0,2)
I_bar = model_fit(cK_bar,params[0][0],params[0][1])

ax.set_xlim([0.35,1.05])
ax.plot(I_bar,cK_bar,label = 'Model')
legend.append('Model')

l  = plt.errorbar(calib_data['Rel. Intes'],calib_data['K conc [mM]'],rel_errorK*calib_data['K conc [mM]'],drelI,c = 'k')

ax.set_xticks([0.4,0.6,0.8,1])
ax.set_xticklabels([0.4,0.6,0.8,1],fontsize  = 15)

ax.set_yticks([0,1,2])
ax.set_yticklabels([0,1,2],fontsize = 15)
ax.set_ylim([-0.1,2.1])
plt.tick_params(direction = 'in',top = True,bottom = True,right = True, left = True,length = 5)
ax.set_xlabel('Rel Intensity [a.u]',fontsize = 18)
ax.set_ylabel('[K]/[mM]',fontsize = 18)
l[0].remove()
plt.legend(legend)



3
2
1
0


In [4]:
fit_dict = {'Fitting Params':fit}
df_fit = pd.DataFrame(fit_dict)

df_fit.to_csv('../Calibrations/Calibration_approx_fit_params.csv')

NameError: name 'fit' is not defined

# Load calibration function parameters and define function


In [4]:
fitsdf = pd.read_csv('../Calibrations/Calibration_approx_fit_params.csv')

fit = fitsdf['Fitting Params']

def fit_func(I,fit):
    
    c = np.zeros_like(I)
    
    for i in range(1,len(fit)+1):

        c += fit[i-1]*(-np.log(I))**(len(fit)-i)
        print(len(fit)-i)
    return c

In [5]:


t0 = 0
t1 = 200
t2 = -1
dt = 9/60

In [52]:
# load 260pM Gramicidin A experiment
DATA_DIR = '/media/marcus/9C33-6BBD/20211005_OmpF_PCPG_KTransport_2/'
dfI = pd.read_csv(os.path.join(DATA_DIR,'collatedIabs.csv'))
dfA = pd.read_csv(os.path.join(DATA_DIR,'collatedAabs.csv'))

In [11]:
%matplotlib qt
fig, ax = plt.subplots(1,1)


for key in dfI.columns[1:]:
    ax.plot(dfI[key])


In [53]:
dfI = dfI.iloc[:,1:]
dfA = dfA.iloc[:,1:]

In [54]:
dfA = dfA.fillna(0)
dfI = dfI.fillna(0)

# Remove small detected objects, most likely not GUVs

**check that the size is appropriate for the experiment**

In [55]:
# remove traces for GUVs below a certain area

App = 3.16 #um^2
Amin= 50*App*1e-12 #m^2

for col in dfI.columns:
    
    if col in dfA.columns:
        area = 1e-12*App*np.mean(dfA[col].iloc[:30])
        
        if area < Amin:
            dfI.pop(col)
            dfA.pop(col)

# Remove intensity curves with sudden drops
 These are likely from GUVs suddenly escaping the trap

In [20]:

def remove_burst_events(data,threshold = 0.3):
    
    for column in data.columns:
        
        if len(data[column][data[column] < threshold]) >=1:
            data.pop(column)
            

In [58]:
%matplotlib qt
fig, ax = plt.subplots(1,1)

for col in dfI.columns:
    ax.plot(dfI[col].iloc[5:])

In [57]:
# normalise intensities

for col in dfI.columns:
    dfI[col] = dfI[col]/np.mean(dfI[col][:30])
remove_burst_events(dfI)

# Smooth data

## Smoothing strategy

1. In experiments where there was relatively fast decay in intensity initially, but slows down (as exponential decays do..), yet there is still a lot of noise, it may be necessary to smooth different regions with different 'cutoff's.
When the intensity has dropped, the signal to noise drops. What's more during the slower period of intensity decay, there is increasing likelihood that some long timescale event will have happened to distort the data. This long timescale noise may be of a similar timescale to the 'signal' in the early period of rapid decay. 

2. The **solution** chosen here, is to have the user select a timepoint to bisect the time traces. The early period up until that timepoint is smoothed with the 1st user input smoothing parameter (usually between 11-21). Then the later period of slow decay is smoother with much larger smoothing settings (50-80). In order to ensure a smooth 'stitching' of these differently smoothed traces, we concatenate and repeat the smoothing with the smallest of the two specified smoothing parameters.

In [59]:
def smooth_decay(I,t0,t1= -1,t2 = -1,smoothing_0 = 19,smoothing_1 = 51,smoothing_2 = 71):
    
    
    if not  smoothing_0 == 0:
        
        if smoothing_0 > t1-t0 and t1 != -1:
            raise ValueError('Smoothing parameter cannot be greater than input data')

        smoothed_decay_0 = I.iloc[t0:t1]

        smoothed_decay_0 = savgol_filter(smoothed_decay_0,smoothing_0,1)
    else:
        smoothed_decay_0 = I.iloc[t0:t1]
    
    if t1 != -1:
        
        smoothed_decay_1 = I.iloc[t1:t2]
        smoothed_decay_1  = savgol_filter(smoothed_decay_1,smoothing_1,1)
        
        smoothed_decay_1 = np.concatenate((smoothed_decay_0,smoothed_decay_1))
    else:
        smoothed_decay_1 = smoothed_decay_0
   
    if t2 != -1:
        smoothed_decay_2 = I.iloc[t2:]
        smoothed_decay_2  = savgol_filter(smoothed_decay_2,smoothing_2,1)
        smoothed_decay_2 = np.concatenate((smoothed_decay_1,smoothed_decay_2))
        
        
    else:
        smoothed_decay_2 = np.append(smoothed_decay_1,[I.iloc[-1]])
        
    smoothed_decay = smoothed_decay_2
    
    if not smoothing_0 == 0:
        final_smoothed_decay = savgol_filter(smoothed_decay,smoothing_0,1)
    else:
        final_smoothed_decay = smoothed_decay
    return final_smoothed_decay
                                  
                                    

In [32]:
#Check smoothing still fits data
#
num = 8
fig,axs = plt.subplots(1,num)

for i,col in enumerate(dfI.columns[np.random.randint(0,len(dfI.columns),num)]):
    
    axs[i].plot(dfI[col].iloc[t0:].to_numpy())
    axs[i].plot(smooth_decay(dfI[col],t0,t1))
    

# Use calibration inverse to convert I to [K]

In [111]:
t0_I = 3
smootheddfI = {}
fig, ax = plt.subplots(1,1)

for col in dfI.columns:
    
    smootheddfI[col] = smooth_decay(dfI[col],t0_I,t1,t2)
    
    ax.plot(smootheddfI[col])
    
smootheddfI = pd.DataFrame(smootheddfI)




In [75]:
# filter columns for plotting

keep = []
for col in smootheddfI.columns:
    
    if len(smootheddfI[col].iloc[smootheddfI[col].to_numpy()>1.1]) > 1:
        continue
    elif len(smootheddfI[col].iloc[smootheddfI[col].to_numpy() < 0.75]) < 1:
        continue
    
    keep.append(col)
    
smootheddfI = smootheddfI[keep]

In [28]:
fig, ax = plt.subplots(1,1)
for col in smootheddfI.columns:
    

    
    ax.plot(smootheddfI[col])

In [29]:
active_data = smootheddfI.copy()
# #filter out vesicles that did nothing
# active_data = data.copy()
#


# active_data= active_data.subtract(background[132:],0)

# for column in active_data.keys():
#     active_data[column] = active_data[column]/np.mean(active_data[column][100:110])
#     active_data[column] = active_data[column]
    
#     if np.std(active_data[column]) > 0.5* np.max(active_data[column]):
#         active_data.pop(column)

#     elif len(active_data[column][active_data[column]< 0.4]) > 1:
#         active_data.pop(column)
     
#     elif len(active_data[column][active_data[column]< 0.75]) < 1:
#         active_data.pop(column)
        
#         continue
#     active_data[column] = active_data[column] - background[132:]

# active_data = active_data.subtract(background[132:],0)
    

mean_kinetics = active_data.mean(axis = 1)


std_kinetics = active_data.std(axis = 1)
lower_quantile = active_data.quantile(0.25,axis = 1)
upper_quantile = active_data.quantile(0.75,axis = 1)


In [24]:
deltaCm = approx_k_conc[25+t0:-1] - fit_func(mean_kinetics,fit)
J_m = R/3*np.gradient(fit_func(mean_kinetics,fit),dt)/60

NameError: name 'approx_k_conc' is not defined

In [322]:
fig,ax = plt.subplots(1,1)

ax.scatter(deltaCm[5:],J_m[5:])
ax.set_ylim([-1e-9,20e-9])

(-1e-09, 2e-08)

In [30]:
mean_kinetics.to_csv('20210427_OmpF_mean_I.csv')
lower_quantile.to_csv('20210427_OmpF_lower_I.csv')
upper_quantile.to_csv('20210427_OmpF_upper_I.csv')

/home/marcus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
/home/marcus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  
/home/marcus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid 

In [323]:
J_mean = pd.DataFrame({'J':J_m})
deltaC_mean = pd.DataFrame({'dC':deltaCm})

In [324]:
J_mean.to_csv('20210427_OmpF_mean_J.csv')
deltaC_mean.to_csv('20210427_OmpF_mean_deltaC.csv')

# Convert intensities to concentrations

In [60]:
dfC_dict = {}

# can't exploit numpy broadcasting

for col in dfI.columns:
    
    smoothed_decay = smooth_decay(dfI[col],t0,t1,t2)
#     print(smoothed_decay)
#     normalised_sd = smoothed_decay/np.mean(smoothed_decay[:30])
    
    cs = []
    for I in smoothed_decay:
#         print(I)
        cs.append(fit_func(I,fit))
    
    
    dfC_dict[col] = cs
    
dfC = pd.DataFrame(dfC_dict)

3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0


2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3


3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0


2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3


3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0
3
2
1
0


In [32]:
# # filter concentration data by gradient
# # if 

# fig, axs = plt.subplots(1,2)

# for col in dfC.columns:
    
#     grad = np.gradient(dfC[col])
    
# #     if len(grad[grad <= -0.0009])/len(grad) > 0.05:
# #         dfC.pop(col)
# #         continue
#     axs[0].hist(grad,15)
#     axs[1].plot(dfC[col])


In [33]:
# find representative background concentration variation

# tifs = glob('/media/marcus/9C33-6BBD/20210920_GramA_260pM07_3nM815_1mMKCl_1/260pM/*.tif')

# bgs = []

# fig, ax = plt.subplots(1,1)

# for tif in tifs:
    
#     vid = tf.TiffFile(tif)
    
#     frames = vid.asarray()
    
#     bg = np.median(frames,axis  =(1,2))
#     bg = np.array(bg)/np.max(bg)
#     bgs.append(bg)
    
    
#     ax.plot(bg)

# This confirms that the normalised background is the same for each chamber. Suggesting potassium concs are identical between chambers



In [61]:
tifs = glob('/media/marcus/9C33-6BBD/20211005_OmpF_PCPG_KTransport_2/20211005_OmpF_PCPG_KTransport_2_MMStack_Pos1.ome.tif')
tif = tifs[-1]

vid = tf.TiffFile(tif)

frames = vid.asarray()
bg = np.median(frames[:,447:512,396:440],axis = (1,2))
# bg = bg[:637]
bg = np.array(bg)-np.mean(bg[-20:-10])
bg = bg/np.max(bg)
bg_k = 1-bg
# make pandas dataframe 

bg= pd.DataFrame({'Bg': bg})
smoothed_bg = smooth_decay(bg['Bg'],0,150,smoothing_0=7,smoothing_1=11)


OME series: not an ome-tiff master file


# Fit to background using diffusion model

In [62]:
t = np.arange(-100*60,100*60,1)
from scipy.special import erf
# parameters

D1 = 4e-10 #m^2s^-1
D2 = 1e-9 #m^2s^-1

Q = 20e-9/3600 #m^3s-1
A = (30e-6)*(800e-6)
R_tube = 1.5e-3 #m
L_tube = 0.15 #m
V = np.pi*R_tube**2*L_tube

def erf_input(D,k,t_c):
    return (Q*(t-t_c)/A)/(np.sqrt(4*np.pi*k*D*V/Q))
z1 = erf_input(D1,0,1)
z2 = erf_input(D2,0,1)
y1 = 0.5*(1+erf(z1))
y2 = 0.5*(1+erf(z2))

def background(t,k,t_c):
    D = 4e-10 #m^2s^-1
    z = erf_input(D,k,t_c)
    
    return_val = 0.5*(1+erf(z))
    
    return return_val

/home/marcus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/home/marcus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


In [63]:

# estimate the offset time by the 0.5 point time
# the relative diffusion to ideal diffusion constant needs to be estimated with experience

p0 = [0.5,34]

t = dt*np.arange(400,bg_k[400:].shape[0]+400)

params = curve_fit(background,t,bg_k[400:],p0)



/home/marcus/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in sqrt
  from ipykernel import kernelapp as app


# use fit to generate background concentration


In [72]:
    
t_0_record = 400+t0
t_bar = np.arange(400,len(frames))
approx_k_conc = background(t_bar,params[0][0],params[0][1])

In [73]:
approx_k_conc = np.concatenate((np.zeros((400,)),approx_k_conc[:len(frames)-400]))

In [75]:
# approx_k_conc = np.concatenate((approx_k_conc,np.ones((37,))))
fig,ax = plt.subplots(1,1)

ax.plot(approx_k_conc)
ax.plot(bg_k)

In [200]:
len(dfC),len(frames)

(293, 321)

In [77]:
%matplotlib qt
fig, ax = plt.subplots(1,1)

for col in dfI.columns:
    ax.plot(np.arange(len(dfC[col])),dfC[col])
    
    
t_0_record = 400+t0
ax.plot(np.arange(len(approx_k_conc[t_0_record:600+t_0_record])),approx_k_conc[t_0_record:600+t_0_record])

In [131]:
# c_bg = []
# for x in bg[360+t0:]:
#     c_bg.append(inv(x))

In [132]:
# fig, axs = plt.subplots(1,2)


# smoothed_background = gaussian(c_bg,sigma)
# axs[0].plot(smoothed_background)
# axs[1].plot(bg[360+t0:])

# Find concentration difference

In [84]:
# calculating c_ext - c_i

dfdeltaC_dict = {}

for col in dfC:
    dfdeltaC = []
    for i in range(len(dfC)):
        
        dfdeltaC.append(approx_k_conc[t_0_record:][i]-dfC[col].iloc[i])
        
    dfdeltaC_dict[col] = dfdeltaC
    
dfdeltaC = pd.DataFrame(dfdeltaC_dict)# drift in the background intensity over time, once the potassium has arrived, is causing a drifting predicted potassium concentration
# this is unlikely to be the case in reality.
# Therefore the best thing may be to just assume that after an initial 80 frame boundary region (20 mins) for this exp
# the potassium concentration externally reaches 1mM.


In [85]:
fig, axC = plt.subplots(1,1)

delete_cols = []

for col in dfdeltaC.columns:
#     graddeltaC = np.gradient(dfdeltaC[col])
    
    
    if len(dfdeltaC[col][dfdeltaC[col].to_numpy() > 1.]) > 1:
        delete_cols.append(col)
        continue
        
    if  len(dfdeltaC[col][dfdeltaC[col].to_numpy() < 0]) > 1:
        delete_cols.append(col)
        continue
    axC.plot(dfdeltaC[col])
axC.plot(approx_k_conc[t_0_record:-1])

In [80]:
for col in delete_cols:#
    dfdeltaC.pop(col)

# Determine the flux

In [86]:
# now calculate J

# J = (1/A)*dN/dt = R/3* dc_i/dt

dfJ_dict = {}


for col in dfdeltaC:
    
    if col in dfA:
        
        delta_c = np.gradient(dfC[col],dt)/60 #molm^-3s^-1
        
        A = np.mean(dfA[col].iloc[:30])*App*1e-12 #m^2
        
        R = np.sqrt(A/np.pi) #m
        
        dfJ_dict[col] = R/3*delta_c #molm^-2s^-1
        

dfJ = pd.DataFrame(dfJ_dict)



# Plot the Flux against concentration gradient



In [87]:
%matplotlib qt
figJ, axJ = plt.subplots(1,1)
figC, axC = plt.subplots(1,1)
figdC,axdC = plt.subplots(1,1)
t_flux_up =0
t_flux_down = 11
c_threshold = 5e-9

def Jfit(deltac,k,c):
    return k*deltac + c

p0 = [1e-14,0]
Pparams = {}

cols = []
for i,col in enumerate(dfJ.columns):#[np.random.randint(0,len(dfJ.columns),5)]):
    
    
    t = dt*np.arange(0,len(dfdeltaC[col]))
    
    J = dfJ[col]
    J_fit = J.iloc[t_flux_up:t_flux_down]
    deltaC = dfdeltaC[col]
    deltaC_fit = deltaC.iloc[t_flux_up:t_flux_down]
    deltaC_bar = np.linspace(dfC[col].min(),1)
    
#     if len(deltaC.iloc[deltaC.to_numpy().astype(float)>0.9])  <3 :
#         dfJ.pop(col)
#         dfdeltaC.pop(col)
#         continue
    params = curve_fit(Jfit,deltaC_fit,J_fit,p0)
    
    J_bar = Jfit(deltaC_fit,*params[0])
    
    P = params[0][0]
    dP = np.sqrt(params[1][0][0])
    print(dP/P, P)
    
#     c = params[0][1]
#     if abs(c) > c_threshold:
#         dfJ.pop(col)
#         dfdeltaC.pop(col)
#         continue
    if P < 0:
        dfJ.pop(col)
        dfdeltaC.pop(col)
        continue
        
    if len(J.iloc[20:].iloc[(deltaC.iloc[20:] < 0.6).to_numpy()]) < 1:
        dfJ.pop(col)
        dfdeltaC.pop(col)
        continue
    
    cols.append(col)
    
    Pparams[col] = params
    
    print(col)
#    plot fluxes
    axJ.scatter(deltaC[t_flux_up:],J[t_flux_up:],alpha = 0.25)
    axJ.plot(deltaC_fit,J_bar)
    
    #plot the Intensity vs time
    
    axdC.plot(t,dfdeltaC[col])
    axC.plot(t,dfC[col])
#Flux plot params


axJ.set_xticks([0,0.5,1])
axJ.set_xlim([-0.05,1.05])
axJ.set_xticklabels([0,0.5,1],fontsize = 15)

axJ.set_xlabel('$[K]_{ext} - [K]_{in}$ [mM]',fontsize = 18)
axJ.set_yticks([0,5e-9,1e-8,1.5e-8])
axJ.set_yticklabels([0,5,10,15],fontsize = 15)

axJ.set_ylim([-1e-9,16e-9])
#  
axJ.set_ylabel('J  [$10^{-12}$ mol$m^{-2}s^{-1}$]',fontsize = 18)
axJ.tick_params(direction = 'in',left = True,right = True, top = True, bottom = True,length = 7)

axC.set_xticks([0,20,40])
axC.set_xticklabels([0,20,40],fontsize = 15)
axC.set_xlabel('Time [mins]',fontsize = 18)

axC.set_yticks([0,0.5,1])
axC.set_yticklabels([0,0.5,1],fontsize = 15)
axC.set_ylabel('Rel I',fontsize = 18 )
axC.set_ylim([-0.05,1.05])
axC.set_xlim([-2,45])

# global params

axC.tick_params(direction = 'in',left = True,right = True, top = True, bottom = True,length = 7)


-0.09663596620873507 -1.765855587083144e-09
-0.45902525104571584 -4.294623600236218e-09
-0.9881367112479944 -9.528411567762013e-10
0.47735308376913116 4.683060977457809e-09
07002
0.4277642230761739 8.718278847515271e-09
07007
-0.11373100650424633 -2.2237665889477166e-09
0.5235716774123258 3.0800394823684727e-09
01016
-0.8991228024844857 -8.067896097712203e-10
-0.09676081140314385 -4.2632946374464526e-09
-0.38547640801598804 -5.414086915484934e-09
0.4429814846030204 4.922380868657372e-09
05016


# Save permeability values

In [143]:
lost = 0
for col in list(Pparams.keys()):
    if Pparams[col][0][1] - abs(np.sqrt(Pparams[col][1][1][1])) < 0:
    #
    
        print(Pparams[col][0][0])
    else:
        print('removed intercept: ',Pparams[col][0][1])
        Pparams.pop(col)
        dfJ.pop(col)
        dfdeltaC.pop(col)
        lost += 1
        
        
print(lost)

removed intercept:  2.7413736198134294e-09
removed intercept:  4.530117070877207e-09
1.2001680936965327e-08
1.3990000772276685e-08
8.885029809171254e-09
removed intercept:  2.3411084949373303e-09
removed intercept:  5.238343846852779e-10
1.5690306949632273e-08
1.025794741617359e-08
6.655978212579282e-08
6.642189896012194e-09
1.7772830552293346e-08
1.2289554123666006e-08
1.5542298409533373e-08
1.4722670682861445e-08
2.320446326537402e-08
removed intercept:  5.183253002222904e-09
removed intercept:  1.243957158827714e-09
2.5435999517655934e-08
1.0798127946428461e-08
2.7145863117089487e-08
removed intercept:  4.593739719627627e-09
removed intercept:  4.1576937209559035e-09
removed intercept:  3.787621211276954e-09
1.53749027239076e-08
1.1613713775094687e-08
1.2405147152409245e-08
1.477456758133039e-08
1.6714313976816524e-08
1.0238136799819856e-08
1.456874739692608e-08
removed intercept:  7.449394518464673e-09
2.6997027342205978e-08
1.0552637657972408e-08
1.2348124068621594e-08
1.621576918

In [144]:
Pparams

{'12002': (array([1.20016809e-08, 4.94014448e-10]),
  array([[ 1.90814581e-18, -1.00653870e-18],
         [-1.00653870e-18,  5.91796488e-19]])),
 '10002': (array([ 1.39900008e-08, -7.42650375e-09]),
  array([[ 2.87338753e-18, -1.62713075e-18],
         [-1.62713075e-18,  1.06730606e-18]])),
 '10004': (array([ 8.88502981e-09, -9.02994510e-10]),
  array([[ 1.03293978e-18, -5.75942951e-19],
         [-5.75942951e-19,  3.62533211e-19]])),
 '09004': (array([ 1.56903069e-08, -9.73616005e-09]),
  array([[ 3.29657528e-18, -1.90425710e-18],
         [-1.90425710e-18,  1.27791974e-18]])),
 '09012': (array([ 1.02579474e-08, -7.24372359e-11]),
  array([[ 2.21397946e-18, -1.24645378e-18],
         [-1.24645378e-18,  7.81798416e-19]])),
 '09014': (array([ 6.65597821e-08, -3.26347545e-08]),
  array([[ 1.50456962e-16, -9.10547111e-17],
         [-9.10547111e-17,  6.05168869e-17]])),
 '09018': (array([6.6421899e-09, 3.2549552e-10]),
  array([[ 9.43330512e-19, -5.43999191e-19],
         [-5.43999191e-19

In [145]:
P_df = {'P ms^-1': [Pparams[key][0][0] for key in Pparams.keys()],
       'dP/P': [np.sqrt(Pparams[key][1][0][0])/Pparams[key][0][0] for key in Pparams.keys()],
       'A':[np.mean(dfA[key].iloc[:30]) for key in Pparams.keys()]}
        
P_identified = {key:[Pparams[key][0][0]] for key in Pparams.keys()}
P_identified = pd.DataFrame(P_identified)
P_df = pd.DataFrame(P_df)

In [146]:
P_df['P cms^-1'] = P_df['P ms^-1']*100

In [154]:
P_df

,P ms^-1,dP/P,A,P cms^-1
0,1.200168e-08,0.115097,140.733333,1.200168e-06
1,1.399000e-08,0.121166,210.666667,1.399000e-06
2,8.885030e-09,0.114388,164.966667,8.885030e-07
3,1.569031e-08,0.115718,214.566667,1.569031e-06
4,1.025795e-08,0.145053,165.133333,1.025795e-06
5,6.655978e-08,0.184287,153.600000,6.655978e-06
6,6.642190e-09,0.146225,193.200000,6.642190e-07
7,1.777283e-08,0.128819,213.066667,1.777283e-06
8,1.228955e-08,0.106780,256.300000,1.228955e-06
9,1.554230e-08,0.112009,217.533333,1.554230e-06


In [155]:
P_df.to_csv('Permeabilities_OmpF_20210427.csv')

# Save Flux vs deltaC data

In [149]:
dfJ.to_csv('Flux_OmpF_20210427.csv')
dfdeltaC.to_csv('deltaC_OmpF_20210427.csv')
P_identified.to_csv('PbyGUVId_OmpF_20210427.csv')

# Mean J vs Mean concentration difference

Need to bin the concentration differences and then average the flux within each bin


In [151]:
deltaCbins = np.linspace(0.35,0.9,80)

decay_start = 5

deltaC_arr = dfdeltaC.to_numpy()[decay_start:]

dfJoverP = dfJ.iloc[decay_start:]

# for col in dfJoverP.columns:
#     dfJoverP[col] = dfJoverP[col].to_numpy()/P_identified[col].iloc[0]
    
    
J_arr = dfJ.iloc[decay_start:].to_numpy()

J_mean = []
deltaC_mean = []
J_scatter = []

for i in range(len(deltaCbins)-1):
    
    deltaC_mean.append(np.mean(deltaC_arr[(deltaC_arr < deltaCbins[i+1]) * (deltaC_arr >deltaCbins[i])]))
    J_mean.append(np.mean(J_arr[(deltaC_arr < deltaCbins[i+1]) * (deltaC_arr >deltaCbins[i])]))
    J_error = np.std(J_arr[(deltaC_arr < deltaCbins[i+1]) * (deltaC_arr >deltaCbins[i])])
    Ns = np.sum((deltaC_arr < deltaCbins[i+1]) * (deltaC_arr >deltaCbins[i]),axis = 0)
    N = len(Ns[Ns > 1])
    J_scatter.append(J_error/np.sqrt(N))
    print(np.sqrt(N))
#  
# upper_bound = np.array(J_mean) + np.array(J_scatter)
# lower_bound = np.array(J_mean) - np.array(J_scatter)

4.795831523312719
5.0
5.0
5.0990195135927845
5.196152422706632
5.385164807134504
5.291502622129181
5.385164807134504
5.385164807134504
5.477225575051661
5.385164807134504
5.477225575051661
5.477225575051661
5.5677643628300215
5.5677643628300215
5.5677643628300215
5.477225575051661
5.744562646538029
5.916079783099616
5.916079783099616
5.830951894845301
5.916079783099616
5.916079783099616
6.082762530298219
5.916079783099616
5.830951894845301
6.0
5.916079783099616
5.916079783099616
5.916079783099616
6.164414002968976
6.244997998398398
6.164414002968976
6.244997998398398
6.4031242374328485
6.48074069840786
6.324555320336759
6.4031242374328485
6.244997998398398
5.830951894845301
5.830951894845301
6.0
5.830951894845301
6.082762530298219
5.830951894845301
5.744562646538029
5.916079783099616
5.5677643628300215
5.656854249492381
5.196152422706632
5.196152422706632
5.744562646538029
5.0990195135927845
5.0990195135927845
5.196152422706632
5.291502622129181
4.898979485566356
4.898979485566356
5.74

In [152]:
figm,axm = plt.subplots(1,1)


axm.scatter(deltaC_mean,J_mean)
upper_bound = np.array(J_mean) + np.array(J_scatter)
lower_bound = np.array(J_mean) - np.array(J_scatter)
axm.fill_between(deltaC_mean,lower_bound,upper_bound, alpha = 0.25,color = 'g')
axm.set_ylim([-1e-9,16e-9])

(-1e-09, 1.6e-08)

In [153]:
meanFluxdata = {'deltaC':deltaC_mean,'J': J_mean,'semJ':J_scatter}

meanFluxdata = pd.DataFrame(meanFluxdata)


meanFluxdata.to_csv('20210427_OmpF_MeanFluxvsdeltaC.csv')



In [97]:
fig, axs = plt.subplots(1,3)
legend = []

for col in dfJ.columns:

    legend.append(col)
#     print(dfdeltaC)

    axs[0].scatter(dfdeltaC[col],dfJ[col]*1e10, marker = '+', s = 10)
    print(dfJ[col])

    axs[1].plot(dt*np.arange(0,len(dfC[col])),dfC[col])

    
#     gradients = []
#     win = 10
#     for i in range(dfJ[col].shape[0]-(win-1)):
        
#         gradient = np.mean(np.gradient(dfJ[col].iloc[i:i+win]))
#         gradients.append(gradient)
        
    
    axs[2].plot(np.gradient(dfJ[col],dfdeltaC[col]))
axs[0].tick_params(direction = 'in',left = True,right = True,bottom = True,top = True,length = 6)
axs[1].tick_params(direction = 'in',left = True,right = True,bottom = True,top = True,length = 6)
axs[0].set_xlabel('[K_ext]-[K_int] [mM]',fontsize = 18)

axs[0].set_ylabel('J [molm^-2s^-1]',fontsize = 18)

axs[1].set_ylabel('[K_int] [mM]',fontsize = 18)


axs[1].set_xlabel(' T [mins]',fontsize = 18)



#xticks

axs[0].set_xticks([0.5,0.75,1])
axs[0].set_xticklabels([0.5,0.75,1],fontsize = 15)

axs[1].set_xticks([0,20,40])
axs[1].set_xticklabels([0,20,40],fontsize = 15)

#yticks

axs[0].set_yticks([0,0.005,0.01,0.015])
axs[0].set_yticklabels([0,0.005,0.01,0.015],fontsize = 15)

axs[0].set_ylim([-0.0005,0.0155])
axs[1].set_yticks([0, 0.25,0.5])
axs[1].set_yticklabels([0,0.25,0.5],fontsize = 15)


# plt.rcParams['text.usetex'] = True

# def myticks(x,pos):

#     if x == 0: return "$0$"

#     exponent = int(np.log10(x))
#     coeff = x/10**exponent

#     return r"${:2.0f} \times 10^{{ {:2d} }}$".format(coeff,exponent)

# axs[0].yaxis.set_major_formatter(ticker.FuncFormatter(myticks))

[Text(0, 0, '0'), Text(0, 0, '0.25'), Text(0, 0, '0.5')]